In [ ]:
import pandas as pd
import pickle
import numpy as np
import xgboost
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, KFold,StratifiedShuffleSplit
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score,roc_auc_score, roc_curve, average_precision_score,precision_recall_curve
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option("display.max_columns",80)

In [ ]:
# Load the training set
df = pd.read_csv('training_dataa/riyadh_training_set.csv')
df = df.dropna(how='any',axis=0)
df.shape

In [ ]:
ohe_fields=['one_way','surface_type','street_type','hour','weekday','month']

# One-Hot encode a couple of variables
df_ohe = pd.get_dummies(df,columns=ohe_fields)

# Get the one-hot variable names
ohe_feature_names = pd.get_dummies(df[ohe_fields],columns=ohe_fields).columns.tolist()
df_ohe.head()

In [ ]:
# Sinuosity is typically close to 1, even for moderately curvy roads. A high sinuosity means a longer road.
feature_transforms = {
    'sinuosity': np.log
}
for feature,transform in feature_transforms.items():
    df_ohe[feature] = transform(df_ohe[feature])

# Continuously valued features
float_feature_names = [
    'accident_counts',
    'speed_limit',
    'aadt',
    'surface_width',
    'sinuosity',
    'euclidean_length',
    'segment_length',
    'road_orient_approx',
    'Rain',
    'dust',
    'temperature',
    'visibility',
    'wind_speed',
    'proximity_to_billboard',
    'proximity_to_major_road',
    'proximity_to_signal',
    'proximity_to_nearest_intersection',
    'proximity_to_nearest_exit',
    'population_density',
    'Hopspot',
    'solar_azimuth',
    'solar_elevation',
]
float_features = df_ohe.xs(float_feature_names,axis=1).values

# Use scikit-learn's StandardScaler
scaler = StandardScaler()
float_scaled = scaler.fit_transform(float_features)
#print (float_features.mean(axis=0))

df_ohe[float_feature_names] = float_scaled

with open('scalers.pkl','wb') as fp:
    pickle.dump(scaler,fp)

In [ ]:
y = df['target'].values

binary_feature_names = [
    'extreme_air_temperature',
    'dew_point_temperature',
    'sky_cover_layer',
    'at_exit',
    'at_intersection',
]

df_ohe = df_ohe.xs(float_feature_names+binary_feature_names+ohe_feature_names,axis=1)

In [ ]:
X = df_ohe.values
y = df['target'].values
feature_names = df_ohe.columns.tolist()

In [ ]:
wrangler = {
    'scaler': scaler,
    'float_scaler_std': float_scaled,
    'float_feature_names': float_feature_names,
    'ohe_fields': ohe_fields,
    'feature_names': feature_names,
    'feature_transforms': feature_transforms 
}
with open('wrangler.pkl','wb') as fp:
    pickle.dump(wrangler,fp)

In [ ]:
feature_sel = range(len(feature_names))
#feature_sel = [-1,-2,-3]
Xs = X[:,feature_sel]
X_train, X_test, y_train, y_test = train_test_split(Xs, y, test_size=0.1)#, random_state=2)
fnames = np.array(feature_names)[feature_sel]

dtrain = xgboost.DMatrix(X_train,label=y_train,feature_names=fnames)
dtest =  xgboost.DMatrix(X_test,label=y_test,feature_names=fnames)


params = {
    'max_depth':6,
    'min_child_weight': 5.0,
    'reg_lambda': 1.0,
    'reg_alpha':0.0,
    'scale_pos_weight':1.0,
    'eval_metric':'auc',
    'objective':'binary:logistic',
    'eta':0.5
}

In [ ]:
booster = xgboost.train(params,dtrain,
    evals = [(dtest, 'eval')],
    num_boost_round=3000,
    early_stopping_rounds=25
)

In [ ]:
print(fnames)

In [ ]:
plt.figure(figsize=(15,15))
xgboost.plot_importance(booster,ax=plt.gca(),importance_type='weight')

In [ ]:
booster.save_model('new_0001.model')

In [ ]:
y_pred_test = booster.predict(dtest)

fpr, tpr, thresholds = roc_curve(y_test,y_pred_test)

y_pred_train = booster.predict(dtrain)
fpr_train, tpr_train, thresholds_train = roc_curve(y_train,y_pred_train)
fig,ax = plt.subplots()
plt.plot([0,1],[0,1],'r-',label='Random Guess',color='orange',lw=3)
plt.plot(fpr,tpr,label='ROC (Test)',lw=3)
plt.plot(fpr_train,tpr_train,'r:',label='ROC (Train)',color='steelblue',lw=3)
plt.grid()
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()

In [ ]:
plt.plot(thresholds,tpr,'r-',label='TPR (Test)',color='orange',lw=3)
plt.plot(thresholds_train,tpr_train,'r:',label='TPR (Train',color='orange',lw=3)
plt.plot(thresholds,fpr,'r-',label='FPR (Test)',color='steelblue',lw=3)
plt.plot(thresholds_train,fpr_train,'r:',label='FPR (Train)',color='steelblue',lw=3)
plt.gca().set_xbound(lower=0,upper=1)
plt.xlabel('Threshold')
plt.ylabel('True/False Positive Rate')
plt.legend()

In [ ]:
plt.figure(figsize=(15,15))

y_pred_test = booster.predict(dtest)
y_pred_train = booster.predict(dtrain)

precision,recall,thresholds = precision_recall_curve(y_test,y_pred_test)
precision_train, recall_train, thresholds_train = precision_recall_curve(y_train,y_pred_train)
fig,ax = plt.subplots()
plt.plot(precision,recall,label='PR (Test)',lw=3)
plt.plot(precision_train,recall_train,label='PR (Train)',lw=3)
plt.xlabel('Precision')
plt.ylabel('Recall')
plt.grid()
plt.legend()
plt.matplotlib.__version__

In [ ]:
plt.plot(thresholds,precision[:-1],'r-',label='P (Test)',color='orange',lw=3)
plt.plot(thresholds_train,precision_train[:-1],'r:',label='P (Train',color='orange',lw=3)
plt.plot(thresholds,recall[:-1],'r-',label='R (Test)',color='steelblue',lw=3)
plt.plot(thresholds_train,recall_train[:-1],'r:',label='R (Train)',color='steelblue',lw=3)
#plt.plot([0,1],[0,1],'k-',lw=2)
plt.gca().set_xbound(lower=0,upper=1)
plt.xlabel('Threshold')
plt.ylabel('Precision/Recall')
plt.legend()

In [ ]:
y_pred_test = booster.predict(dtest) > 0.19
print ('Test Accuracy:',accuracy_score(y_test,y_pred_test))
print ('Test F1:',f1_score(y_test,y_pred_test))
print ('Test Precision:',precision_score(y_test,y_pred_test))
print ('Test Recall:',recall_score(y_test,y_pred_test))
y_pred_test = booster.predict(dtest)
print ('Test AUC:',roc_auc_score(y_test,y_pred_test))
print ('Test AP:',average_precision_score(y_test,y_pred_test))

y_pred_train = booster.predict(dtrain) > 0.19
print ('Train Accuracy:',accuracy_score(y_train,y_pred_train))
print ('Train F1:',f1_score(y_train,y_pred_train))
print ('Train Precision:',precision_score(y_train,y_pred_train))
print ('Train Recall:',recall_score(y_train,y_pred_train))
y_pred_train = booster.predict(dtrain)
print ('Train AUC:',roc_auc_score(y_train,y_pred_train))
print ('Test AP:',average_precision_score(y_train,y_pred_train))

In [ ]:
def plot_split_histogram(feature_name):
    hist = booster.get_split_value_histogram(feature_name)
    try:
        i = float_feature_names.index(feature_name)
        fake_data = np.zeros((hist.Count.size,len(float_feature_names)))
        fake_data[:,i] = hist.SplitValue
        hist.loc[:,'SplitValue'] = scaler.inverse_transform(fake_data)[:,i]
    except: pass
    hist.plot(kind='area',x='SplitValue',y='Count')

In [ ]:
plot_split_histogram('temperature')

In [ ]:
plot_split_histogram('solar_azimuth')

In [ ]:
plot_split_histogram('solar_elevation')

In [ ]:
plot_split_histogram('proximity_to_billboard')

In [ ]:
plot_split_histogram('population_density')

In [ ]:
plot_split_histogram('sinuosity')

# plot results

In [ ]:
import pandas as pd
import geopandas as gpd
import time
import pickle
import os
import numpy as np
import xgboost
import pytz
import arcgis
#
#plotting
#'''
from IPython.display import HTML, display
import datashader as ds
from datashader import transfer_functions as tf
from datashader.colors import colormap_select, Greys9, Hot, viridis, inferno
#'''

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import NullFormatter
import matplotlib as mpl
mpl.rc('xtick', color='k')
mpl.rc('ytick', color='k')
%matplotlib inline
#'''
import datetime

In [ ]:
with open('wrangler.pkl','rb') as fp:
    wrangler = pickle.load(fp)

float_scaler_mean = wrangler['scaler']
float_scaler_std = wrangler['float_scaler_std']
float_feature_names = wrangler['float_feature_names']
ohe_fields = wrangler['ohe_fields']
feature_names = wrangler['feature_names']   
booster = xgboost.Booster()
booster.load_model('new_0001.model')

In [ ]:
project_gdb = r'D:\Model_R\Model_RT\Model_RT.gdb'
collisions_path = os.path.join(project_gdb,'collisions_joined_1')
road_features_path = os.path.join(project_gdb,'SRF')

In [ ]:
R_tz = pytz.timezone('Asia/Riyadh')

In [ ]:
road_features = pd.read_csv('training_dataa/road_features.csv')

In [ ]:
collisions =  pd.read_csv('training_dataa/collisions.csv')

In [ ]:
tidx = pd.DatetimeIndex(collisions['timestamp']).floor('H')
tidx = tidx.tz_localize(R_tz,ambiguous='NaT')

In [ ]:
#collisions['timestamp'] = pd.to_datetime(collisions.timestamp).map(utah_tz.localize)
collisions = collisions.set_index(tidx)
collisions.sort_index(inplace=True)

In [ ]:
collisions['hour'] = collisions.index.hour
collisions['weekday'] = collisions.index.weekday
collisions['month'] = collisions.index.month

In [ ]:
wdf = pd.read_csv('Riyadh_weather_2014-2019_grouped.csv')

In [ ]:
wdf['timestamp'] = pd.to_datetime(wdf.timestamp).map(pytz.utc.localize)
wdf['timestamp'] = wdf['timestamp'].map(lambda x: x.astimezone(R_tz))

In [ ]:
wdf = wdf.set_index('timestamp')

In [ ]:
road_features.head()

In [ ]:
import geopandas as gpd
import json
import numpy as np
from shapely.geometry import LineString, Point, box, mapping
import ast
from pyproj import Proj

In [ ]:
paths = road_features.SHAPE.map(lambda x: np.array(ast.literal_eval(x)['paths'][0]))

In [ ]:
pathLineStrings = paths.map(LineString)

In [ ]:
gdf = gpd.GeoDataFrame(road_features,geometry=pathLineStrings)
gdf.crs = {'init': 'epsg:32638'}
#gdf.crs = {'init': 'epsg:4326'}

In [ ]:
gdf.head()

In [ ]:
gdf = gdf.to_crs({'init': 'epsg:4326'})

In [ ]:
x0 = 900714804574
x1 = -5120900
y0 = 900709927374
y1 = -9998100
SLC = box(x0,y0,x1,y1)

In [ ]:
slc_df = gdf[gdf.intersects(SLC)]

In [ ]:
slc_df.head()

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches((10,10))
#help(gdf.plot)
slc_df['scaled'] = slc_df['accident_counts']
slc_df.plot(ax=ax,column='scaled',scheme='quantiles')

In [ ]:
slc_df.head()

In [ ]:
predTimest = pd.date_range('5/22/2016', periods=7*24, freq='H',tz='Asia/Riyadh')

In [ ]:
predTimest

In [ ]:
prediction_time = predTimest[11]

In [ ]:
test_df = slc_df.copy()

In [ ]:
test_df['timestamp'] = prediction_time
test_df['station_id'] = slc_df.station_id.astype('int64')
test_df['hour'] = prediction_time.hour
test_df['weekday'] = prediction_time.weekday()
test_df['month'] = prediction_time.month

In [ ]:
def add_join_key(df):
    df['join_key'] = df.station_id.map(int).map(str)+df.timestamp.map(datetime.datetime.isoformat)
    df = df.set_index('join_key')
    return df

In [ ]:
weath_df = wdf.loc[prediction_time]

In [ ]:
test_df = add_join_key(test_df)
weath_df = add_join_key(weath_df.reset_index())

In [ ]:
test_df = test_df.join(weath_df.drop(columns=['station_id','timestamp']))

In [ ]:
test_df.columns

In [ ]:
def make_test_set(df,wrangler):
    float_scaler_mean = wrangler['scaler']
    float_scaler_std = wrangler['float_scaler_std']
    float_feature_names = wrangler['float_feature_names']
    ohe_fields = wrangler['ohe_fields']
    feature_names = wrangler['feature_names'] 
    print(len(feature_names))
    df_ohe = pd.get_dummies(df,columns=ohe_fields)

    float_features = df.xs(float_feature_names,axis=1).values
    float_features = (float_features - float_scaler_mean) / float_scaler_std
    for i,fname in enumerate(float_feature_names):
        df_ohe[fname] = float_features[:,i]
        
    empty_features = list(set(feature_names) - set(df_ohe.columns.tolist()))
    for f in empty_features:
        df_ohe[f] = 0
    df_ohe = df_ohe[feature_names]
    
    #print(df_ohe.columns)
    #print(df_ohe.columns.tolist())
    X = df_ohe.values
    feature_names = df_ohe.columns.tolist()
    return X, feature_names

In [ ]:
import matplotlib.colors as c
help(c.LinearSegmentedColormap.from_list)
cmap = c.LinearSegmentedColormap.from_list('traffic',['g','g','g','y','orange','r','darkred'])

In [ ]:
'''
%matplotlib
fig,ax = plt.subplots()
for i,pt in enumerate(predTimest[:72]):
    test_df = slc_df.copy()
    test_df['timestamp'] = pt
    test_df['station_id'] = slc_df.station_id.astype('int64')
    test_df['hour'] = prediction_time.hour
    test_df['weekday'] = prediction_time.weekday()
    test_df['month'] = prediction_time.month
    
    weath_df = wdf.loc[pt]
    test_df = add_join_key(test_df)
    weath_df = add_join_key(weath_df.reset_index())
    
    test_df = test_df.join(weath_df.drop(columns=['station_id','timestamp']))
    X,names = make_test_set(test_df,wrangler)
    xm = xgboost.DMatrix(X[:,:],feature_names=names)
    
    pred = booster.predict(xm)
    test_df['probability'] = np.minimum(pred,0.50)
    test_collisions = collisions[(pt - datetime.timedelta(seconds=0)).isoformat():(pt + datetime.timedelta(seconds=3600)).isoformat()]
    fig,ax = plt.subplots()
    
    fig.set_size_inches((15,15))
    test_df.plot(ax=ax,column='probability',cmap=cmap,linewidth=3,alpha=1)
    plt.gca().set_facecolor('k')
    #plt.imshow(np.array([[test_df.probability.min(),test_df.probability.max()]]),origin='lower')
    test_collisions.plot.scatter(x='DDLon',y='DDLat',ax=ax,s=100,marker='*',color='r',zorder=6e99)
    ax.set_xbound(lower=x0,upper=x1)
    ax.set_ybound(lower=y0,upper=y1)
    #plt.colorbar()
    plt.savefig('{}.png'.format(i))
   
'''

In [ ]:
X,names = make_test_set(test_df,wrangler)
print (X.shape)
print (X[0])

In [ ]:
xm = xgboost.DMatrix(X[:,:],feature_names=names[:])

In [ ]:
pred = booster.predict(xm)

In [ ]:
plt.figure()
plt.hist(pred,50)

In [ ]:
test_df['probability'] = pred*100

In [ ]:
test_collisions = collisions[(prediction_time - datetime.timedelta(seconds=0)).isoformat():(prediction_time + datetime.timedelta(seconds=3600)).isoformat()]

In [ ]:
test_df['vis'] = np.log(test_df['probability']) - np.log( test_df.geometry.length)

In [ ]:
test_df['probability'] = np.minimum(pred,0.08)

fig,ax = plt.subplots()
fig.set_size_inches((15,15))
test_df.plot(ax=ax,column='probability',cmap=cmap)
plt.gca().set_facecolor('k')
plt.imshow(np.array([[test_df.probability.min(),test_df.probability.max()]]),origin='lower',cmap=cmap)
test_collisions.plot.scatter(x='Longitude',y='Latitude',
                             ax=ax,
                             s=500,
                             color='w',
                             zorder=9e99,
                             marker='*',
                             edgecolors='r')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
ax.set_xbound(lower=x0,upper=x1)
ax.set_ybound(lower=y0,upper=y1)
plt.colorbar(cax=cax)

In [ ]:
test_df.loc[:,'timestamp'] = test_df.loc[:,'timestamp'].astype(str)

In [ ]:
test_df.to_file('Riyadh_probability.shp', driver='ESRI Shapefile')